So far, this is just a proof of concept for using the API. Obviously, we need to streamline the process of pulling more rows, loading/saving the csv as well as cleaning the data. 

In [2]:
%pip install requests keyring

Note: you may need to restart the kernel to use updated packages.


In [3]:
from cdg_client import CDGClient
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

In [42]:
API_KEY_PATH = '../../../api_key.txt'       # put your api_key path here
                                            # it shouldn't go to github
LIMIT = 250 
parse_xml = lambda data: ET.fromstring(data)


with open(API_KEY_PATH) as file:
    api_key = file.readline()
client = CDGClient(api_key, response_format='xml')
data, _ = client.get(f'bill?format=xml&limit={100}&fromDateTime=2023-01-01T00:00:00Z&toDateTime=2023-01-11T23:59:59Z')

In [43]:
bills_dict = xmltodict.parse(data)['api-root']
bills_dict

{'bills': {'bill': [{'congress': '117',
    'type': 'SRES',
    'originChamber': 'Senate',
    'originChamberCode': 'S',
    'number': '673',
    'url': 'https://api.congress.gov/v3/bill/117/sres/673?format=xml',
    'title': 'A resolution authorizing the Committee on Rules and Administration to prepare a revised edition of the Standing Rules of the Senate as a Senate document.',
    'updateDateIncludingText': '2023-01-10',
    'latestAction': {'actionDate': '2022-06-14',
     'text': 'Submitted in the Senate, considered, and agreed to without amendment by Unanimous Consent. (consideration: CR S2936; text: CR S2944)'},
    'updateDate': '2023-01-10'},
   {'congress': '117',
    'type': 'HRES',
    'originChamber': 'House',
    'originChamberCode': 'H',
    'number': '1159',
    'url': 'https://api.congress.gov/v3/bill/117/hres/1159?format=xml',
    'title': 'Electing Members to certain standing committees of the House of Representatives.',
    'updateDateIncludingText': '2023-01-06',
 

{'bill': [{'congress': '119',
   'type': 'S',
   'originChamber': 'Senate',
   'originChamberCode': 'S',
   'number': '318',
   'url': 'https://api.congress.gov/v3/bill/119/s/318?format=xml',
   'title': 'ANCHOR Act',
   'updateDateIncludingText': '2025-10-11',
   'latestAction': {'actionDate': '2025-10-10',
    'text': 'Held at the desk.',
    'actionTime': '12:33:05'},
   'updateDate': '2025-10-11'},
  {'congress': '119',
   'type': 'S',
   'originChamber': 'Senate',
   'originChamberCode': 'S',
   'number': '269',
   'url': 'https://api.congress.gov/v3/bill/119/s/269?format=xml',
   'title': 'Ending Improper Payments to Deceased People Act',
   'updateDateIncludingText': '2025-10-11',
   'latestAction': {'actionDate': '2025-10-10',
    'text': 'Held at the desk.',
    'actionTime': '12:33:05'},
   'updateDate': '2025-10-11'},
  {'congress': '119',
   'type': 'S',
   'originChamber': 'Senate',
   'originChamberCode': 'S',
   'number': '254',
   'url': 'https://api.congress.gov/v3/bil

In [93]:
bills = xmltodict.parse(data)['api-root']['bills']['bill']
df = pd.DataFrame(bills)
df.head()

,congress,type,originChamber,originChamberCode,number,url,title,updateDateIncludingText,latestAction,updateDate
0,119,SRES,Senate,S,420,https://api.congress.gov/v3/bill/119/sres/420?...,A resolution supporting the designation of Sep...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Submitte...",2025-09-30
1,119,SRES,Senate,S,418,https://api.congress.gov/v3/bill/119/sres/418?...,A resolution expressing support for the design...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Resoluti...",2025-09-30
2,119,S,Senate,S,2144,https://api.congress.gov/v3/bill/119/s/2144?fo...,A bill to improve the safety and security of M...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Passed S...",2025-09-30
3,119,S,Senate,S,1333,https://api.congress.gov/v3/bill/119/s/1333?fo...,Strengthening Child Exploitation Enforcement Act,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Passed S...",2025-09-30
4,119,HR,House,H,5371,https://api.congress.gov/v3/bill/119/hr/5371?f...,"Continuing Appropriations and Extensions Act, ...",2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Cloture ...",2025-09-30


In [94]:
# get the next LIMIT # of bills and add it to the data frame over and over
for i in range(1, 5):
    offset = i * LIMIT
    more_data, _ = client.get(f'bill?format=json&limit={LIMIT}&offset={offset}') 
    more_bills = xmltodict.parse(more_data)['api-root']['bills']['bill']
    more_rows = pd.DataFrame(more_bills)
    df = pd.concat([df, more_rows], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   congress                 50 non-null     object
 1   type                     50 non-null     object
 2   originChamber            50 non-null     object
 3   originChamberCode        50 non-null     object
 4   number                   50 non-null     object
 5   url                      50 non-null     object
 6   title                    50 non-null     object
 7   updateDateIncludingText  50 non-null     object
 8   latestAction             50 non-null     object
 9   updateDate               50 non-null     object
dtypes: object(10)
memory usage: 4.0+ KB


In [95]:
# save as csv 
# # probably want to go to dropbox or something, not directly in the repository
df.to_csv('congress_bills.csv') 